This notebook provides an example workflow of bin2cell, with corresponding results presented.

In [1]:
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import os

import bin2cell as b2c

In [ ]:
## READ-IN DATA, LOAD COUNT MATRIX
#create directory for stardist input/output files
os.makedirs("stardist", exist_ok=True)

path = "../Visium/binned_outputs/square_002um"
source_image_path = "../Visium/Visium_HD_Mouse_Brain_tissue_image.tif"
spaceranger_image_path = "../Visium/spatial"

adata = b2c.read_visium(path, 
                        source_image_path = source_image_path, 
                        spaceranger_image_path = spaceranger_image_path
                       )
adata.var_names_make_unique()
adata

anndata.py (1756): Variable names are not unique. To make them unique, call `.var_names_make_unique`.
anndata.py (1756): Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 6296688 × 19059
    obs: 'in_tissue', 'array_row', 'array_col'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

: 

In [ ]:
## FILTER DATA
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_counts=1)
## CONTROL RESOLUTION OF INPUT IMAGE
mpp = 0.3
b2c.scaled_he_image(adata, mpp=mpp, save_path="stardist/he.tiff") # image needs to be saved to the drive

In [ ]:
## DESTRIPE DATA, CROP, and PLOT
b2c.destripe(adata)
mask = ((adata.obs['array_row'] >= 1450) & 
        (adata.obs['array_row'] <= 1550) & 
        (adata.obs['array_col'] >= 250) & 
        (adata.obs['array_col'] <= 450)
       )
bdata = adata[mask]

# Jupyter issue when plotting
#sc.pl.spatial(bdata, color=[None, "n_counts", "n_counts_adjusted"], color_map="OrRd",
              #img_key="0.3_mpp_150_buffer", basis="spatial_cropped_150_buffer")

_construct.py (149): Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


In [ ]:
## SEGMENTATION
b2c.stardist(image_path="stardist/he.tiff", 
             labels_npz_path="stardist/he.npz", # define the path where segmentation label is saved
             stardist_model="2D_versatile_he", 
             prob_thresh=0.01 # probability threshold: A lower threshold (0.01) means StarDist will detect more nuclei, including lower-confidence ones.
            )

b2c.insert_labels(adata, 
                  labels_npz_path="stardist/he.npz", # path to the labels generated by stardist
                  basis="spatial", 
                  spatial_key="spatial_cropped_150_buffer", # the key where the cropped image is stored
                  mpp=mpp, 
                  labels_key="labels_he" # where segmentation labels will be stored
                 )

# visualize
bdata = adata[mask]

#the labels obs are integers, 0 means unassigned
bdata = bdata[bdata.obs['labels_he']>0] # assigned labels
bdata.obs['labels_he'] = bdata.obs['labels_he'].astype(str)

# plot, see on website
# sc.pl.spatial(bdata, color=[None, "labels_he"], img_key="0.5_mpp_150_buffer", basis="spatial_cropped_150_buffer") # basis = spatial coordinate basis

2025-02-13 16:39:35.170700: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 16:39:35.187707: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-13 16:39:35.286797: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-13 16:39:35.420589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739482775.510383 1201886 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739482775.51

: 